In [2]:
import numpy as np 
import pandas as pd
import requests
from groq import Groq
from datetime import datetime
import calendar
import matplotlib.pyplot as plt
import math
from openai import OpenAI
import time

In [3]:
client = Groq(api_key="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")

In [31]:
def getTickerLlma(company):
    response=""
    completion = client.chat.completions.create(
        model="llama3-groq-70b-8192-tool-use-preview",
        messages=[
            {
                "role": "system",
                "content": "You are a stock analyst identifying whether a given company, or its parent companies and subsidiaries, \"\n                    \"are publicly traded.\\n\\n# Required Information:\\n- Given company name.\\n\\n# Details to Include:\\n\"\n                    \"- Include parent or subsidiary companies of the given company, if they are publicly traded.\\n\"\n                    \"- Include the given company itself if it is publicly traded.\\n\\n# Output Format:\\n\"\n                    \"Provide a list of the companies separated by a comma of their TICKERS ONLY. Do not include further information \"\n                    \"or notes. Do not output anything more than just the ticker, no explenation, no description, only the ticker. Output 'none' if the company is not publicly traded.\""
            },
            {
                "role": "user",
                "content": company
            },
        ],
        temperature=0.5,
        max_tokens=1024,
        top_p=0.65,
        stream=True,
        stop=None,
    )

    for chunk in completion:
        if chunk.choices[0].delta and chunk.choices[0].delta.content:
            response+=(chunk.choices[0].delta.content)
    return response


In [60]:
contracts = pd.read_csv("/home/xikron/Projects/misc/data/contracts_data.csv")
contracts_test = contracts[contracts["Start Date"].str.match(r"^\d{4}-\d{2}-\d{2}$", na=False)]
contracts_test["Start Date"] = pd.to_datetime(contracts_test["Start Date"])
contracts_test = contracts_test.sort_values(by="Start Date")

/tmp/ipykernel_4875/3023700321.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contracts_test["Start Date"] = pd.to_datetime(contracts_test["Start Date"])


In [61]:
blacktest_contracts = contracts_test[
    (contracts_test["Start Date"] >= pd.Timestamp("2010-01-01")) & 
    (contracts_test["Start Date"] < pd.Timestamp("2015-01-01"))
]
covid_contracts_2020 = contracts_test[
    (contracts_test["Start Date"]>= pd.Timestamp("2020-01-01")) & 
    (contracts_test["Start Date"]< pd.Timestamp("2021-01-01"))
] 
covid_contracts_2021 = contracts_test[
    (contracts_test["Start Date"]>= pd.Timestamp("2021-01-01")) & 
    (contracts_test["Start Date"]< pd.Timestamp("2022-01-01"))
] 
covid_contracts_2022 = contracts_test[
    (contracts_test["Start Date"]>= pd.Timestamp("2022-01-01")) & 
    (contracts_test["Start Date"]< pd.Timestamp("2023-01-01"))
] 

In [69]:
sContracts = blacktest_contracts.sample(1500, replace=False).reset_index(drop=True, inplace=False)
sCC20 = covid_contracts_2020.sample(500, replace=False).reset_index(drop=True, inplace=False)
sCC21 = covid_contracts_2021.sample(500, replace=False).reset_index(drop=True, inplace=False)
sCC22 = covid_contracts_2022.sample(500, replace=False).reset_index(drop=True, inplace=False)

In [70]:
sContracts.head()

,internal_id,Award ID,Recipient Name,Award Amount,Total Outlays,Description,Contract Award Type,def_codes,COVID-19 Obligations,COVID-19 Outlays,...,Infrastructure Outlays,Awarding Agency,Awarding Sub Agency,Start Date,End Date,recipient_id,prime_award_recipient_id,awarding_agency_id,agency_slug,generated_internal_id
0,22891069,HC02,"URS GROUP, INC.",140673803.16,NaN,CONSTRUCTION OF THE AEGIS ASHORE MISSILE DEFEN...,DO,NaN,NaN,NaN,...,NaN,Department of Defense,Department of the Navy,2012-06-04,2013-11-07,0c2a7dab-a206-9d45-c51a-a869e366c2d8-C,NaN,1173,department-of-defense,CONT_AWD_HC02_9700_N6274209D1174_9700
1,15070206,FA868214C0069,LOCKHEED MARTIN CORP,238744809.78,NaN,JOINT AIR-TO-SURFACE STANDOFF MISSILE - EXTEND...,DEFINITIVE CONTRACT,NaN,NaN,NaN,...,NaN,Department of Defense,Defense Contract Management Agency,2013-12-19,2017-09-30,695d5739-ecc3-8484-ba2b-f9c8b4652a4a-C,NaN,1173,department-of-defense,CONT_AWD_FA868214C0069_9700_-NONE-_-NONE-
2,27871346,SAQMMA12F1198,CGI FEDERAL INC.,40116642.16,NaN,GLOBAL SUPPORT STRATEGY FOR VISA SERVICES-EURECA,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of State,Department of State,2012-04-23,2012-04-16,d86df7af-f6c1-a07a-ccc4-ec74331bafc5-C,NaN,315,department-of-state,CONT_AWD_SAQMMA12F1198_1900_SAQMMA10D0018_1900
3,37392580,W31P4Q12C0079,RAYTHEON COMPANY,52339022.34,NaN,THIS EFFORT IS FOR THE UPGRADE OF PATRIOT ADVA...,DEFINITIVE CONTRACT,NaN,NaN,NaN,...,NaN,Department of Defense,Department of the Army,2011-12-23,2015-02-28,cce9df51-3950-87f4-0b34-4353d5a08f32-C,NaN,1173,department-of-defense,CONT_AWD_W31P4Q12C0079_9700_-NONE-_-NONE-
4,37449582,W56HZV10C0263,BAE SYSTEMS INFORMATION & ELECTRONIC SYSTEMS I...,65482441.31,NaN,CHECK-6 CAMERAS FOR CI ARMY,DCA,NaN,NaN,NaN,...,NaN,Department of Defense,Department of the Army,2010-02-26,2011-12-31,8e6846f9-b4e6-7c1a-d0d9-a53eb2bc9499-C,NaN,1173,department-of-defense,CONT_AWD_W56HZV10C0263_9700_-NONE-_-NONE-


In [71]:
contractColumns = contracts_test.columns
apiCalls =0

In [72]:
def getPublicContracts(df): 
    global contractColumns, apiCalls
    publicContracts =[]
    start = time.time()
    for i in range (len(df)):
        if apiCalls>30:
            end = time.time()
            intv = end - start 
            if intv < 60:
                time.sleep(100 - intv)
            
            start = time.time()
            apiCalls = 0
        ticker = getTickerLlma(df.iloc[i]["Recipient Name"])
        apiCalls+=1
        if ticker != "none": 
            df.loc[i, "Recipient Name"] = ticker
            publicContracts.append(df.iloc[i])
    
    return pd.DataFrame(publicContracts, columns=contractColumns).reset_index(drop=True, inplace=False)

In [73]:
pCC20=getPublicContracts(sCC20)

In [74]:
pCC21=getPublicContracts(sCC21)

In [75]:
pCC22=getPublicContracts(sCC22)

In [76]:
pC=getPublicContracts(sContracts)

KeyboardInterrupt: 

In [121]:
grouped_pCC22 = {recipient: group for recipient, group in pCC22.groupby('Recipient Name')}
sorted_keys = sorted(grouped_pCC22.keys(), key=lambda x: len(grouped_pCC22[x]), reverse=True)
sorted_keys[:5]

['BA', 'LMT', 'RTN', 'LDOS', 'LHX']

In [88]:
def setParams(ticker, itv, date):
    params = {
        "function": "TIME_SERIES_INTRADAY", 
        "symbol" : ticker, 
        "interval" : itv, 
        "adjusted" : "false", 
        "month" : date, 
        "outputsize":"full", 
        "datatype" : "json", 
        "extended_hours": "false",
        "apikey": "LYC4QD75RUZISEOV"
    }
    return params 

In [89]:
alphV = "https://www.alphavantage.co/query"

In [156]:
def getHistoric(tik, interval, month):
    global keyCalls, currentKey
    historic = requests.get(alphV, params=setParams(tik, interval, month))
    key = f"Time Series ({interval})"
    if key not in historic.json():
        return pd.DataFrame() 
    else: 
        df = pd.DataFrame.from_dict(historic.json()[key]).transpose().reset_index().rename(columns={'index':'date'})
        df.columns = df.columns.str.replace(r'^\d+\.\s*', '', regex=True)
        df['date'] = pd.to_datetime(df['date'])
        df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
        return df.sort_values(by='date', ascending= True).reset_index(drop=True)

In [97]:
def getYTDData(tik, interval, contractDate):
    date = datetime.strptime(contractDate, "%Y-%m-%d")
    
    months = []
    for month_offset in range(-1, 13):
        month_date = (date.replace(day=1) + pd.DateOffset(months=month_offset))
        months.append(month_date.strftime("%Y-%m"))
    
    data_frames = []
    for month in months:
        df = getHistoric(tik, interval, month)
        if not df.empty:
            data_frames.append(df)
    
    if data_frames:
        return pd.concat(data_frames, ignore_index=True, sort=False)
    else:
        return pd.DataFrame()

In [161]:
lmTDf = grouped_pCC22["LMT"]
rtxDf = grouped_pCC22["RTN"]
ldosDf = grouped_pCC22["LDOS"]
lhxDf = grouped_pCC22["LHX"]
baDf = grouped_pCC22["BA"]

In [124]:
lmtINTD = getYTDData("LMT", "60min", "2022-01-01")

API called
API called
API called
API called
API called
API called
API called
API called
API called
API called
API called
API called
API called
API called


In [152]:
import pandas as pd
import matplotlib.pyplot as plt
import math

graph = 2022
# Function to generate graph
def genGraph(tik, lmtINTD, lmtDf):
    global graph
    
    # Ensure lmtINTD is for the full year of interest
    lmtINTD['date'] = pd.to_datetime(lmtINTD['date'])  # Ensure date is in datetime format
    lmtINTD_2022 = lmtINTD[lmtINTD['date'].dt.year == 2022]

    plt.figure(figsize=(20, 10))
    
    # Plot the intraday close price for LMT stock
    plt.plot(lmtINTD_2022['date'], lmtINTD_2022['close'], label='Close Price', color='blue')

    # Extract the contract trading days from the lmtDf dataframe
    lmtDf['Start Date'] = pd.to_datetime(lmtDf['Start Date'])  # Ensure date is in datetime format
    contract_dates = lmtDf[lmtDf['Start Date'].dt.year == 2022]['Start Date'].dt.date.unique()  # Unique contract start dates in 2020

    # Plot red markers on the contract dates for each contract
    for date in contract_dates:
        # Get the corresponding close price for each contract start date
        contract_close_price = lmtINTD_2022[lmtINTD_2022['date'].dt.date == date]['close'].iloc[0] if not lmtINTD_2022[lmtINTD_2022['date'].dt.date == date].empty else None
        if contract_close_price is not None:
            plt.scatter(date, contract_close_price, color='red', label=f'Contract on {date}')

    # Customize the plot with axis limits, labels, and title
    x_min = lmtINTD_2022['date'].min().strftime("%Y-%m-%d")
    x_max = lmtINTD_2022['date'].max().strftime("%Y-%m-%d")
    y_min = lmtINTD_2022['close'].min()
    y_max = lmtINTD_2022['close'].max()
    y_ticks = list(range(math.ceil(y_min - 1), math.ceil(y_max + 1.5), math.ceil(abs(y_max - y_min) / 5)))
    plt.yticks(y_ticks)

    # Format the x-axis with the date labels
    plt.xticks(rotation=45, ha='right')
    
    # Add labels and title
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.title(f'{tik} Close Price for 2020')
    
    # Show grid and legend
    plt.grid()
    plt.legend()
    
    # Save the plot as a PDF file
    outputFile = f"/home/xikron/Projects/misc/graphs/{tik}_{str(graph)}.pdf"
    graph += 1
    plt.tight_layout()
    plt.savefig(outputFile, format='pdf')
    plt.close()

# Function to get contract interval based on the contract date
def getContractInterval(data, contractDate):
    return data[data['date'].dt.strftime('%Y-%m-%d').str.contains(getTradingDay(contractDate))]

# Function to get the adjusted trading day (business day) for a given contract date
from pandas.tseries.offsets import CustomBusinessDay
from pandas.tseries.holiday import USFederalHolidayCalendar

def getTradingDay(contractDate):
    us_business_day = CustomBusinessDay(calendar=USFederalHolidayCalendar())
    contractDate = pd.Timestamp(contractDate)
    adjDate = contractDate if us_business_day.is_on_offset(contractDate) else contractDate + us_business_day
    return adjDate.strftime('%Y-%m-%d')


In [167]:
lhxINTD = getYTDData("LHX", "60min", "2022-01-01")
lhxINTD.shape
genGraph("LHX", lhxINTD, lhxDf)

In [166]:
genGraph("LDOS", ldosINTD, ldosDf)

In [164]:
genGraph("RTX", rtxINTD, rtxDf)

In [118]:
lhxINTD = getYTDData("LHX", "60min", "2020-01-01")

API called
API called
API called
API called
API called
API called
API called
API called
API called
API called
API called
API called
API called
API called


In [153]:
genGraph("LMT", lmtINTD, lmTDf)

In [126]:
lmtINTD

,date,open,high,low,close,volume
0,2021-12-01 10:00:00,334.290,335.34,333.01,333.790,169551.0
1,2021-12-01 11:00:00,333.880,335.33,333.12,335.320,193659.0
2,2021-12-01 12:00:00,335.140,335.95,333.42,333.770,184512.0
3,2021-12-01 13:00:00,333.740,334.11,331.06,331.130,133133.0
4,2021-12-01 14:00:00,331.060,331.80,330.36,331.240,125500.0
...,...,...,...,...,...,...
1752,2023-01-31 11:00:00,460.770,461.66,459.01,461.560,68054.0
1753,2023-01-31 12:00:00,461.655,463.23,460.83,461.590,100246.0
1754,2023-01-31 13:00:00,461.590,461.75,460.26,460.580,35328.0
1755,2023-01-31 14:00:00,460.700,462.02,459.53,461.807,87463.0


In [117]:
genGraph("GD", gdINTD, gdDf)

In [120]:
genGraph("LHX", lhxINTD, lhxDf)

In [147]:
print(lmTDf.shape)
lmTDf=lmTDf["Start Date"].dropna()
print(lmTDf.shape)

(22, 21)
(22,)
